In [2]:
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import copy
import glob
import os
import re
import itertools

In [57]:
## Import all files located in the data/FCC_Broadband/used folder adjacent to this directory
## Dict key - the unique part of the file name : Dict value - the dataframe
fcc_path = r'../data/FCC_Broadband/TN_Only'
fcc_files = glob.glob(os.path.join(fcc_path, "*.csv"))
print(fcc_files)
fcc_dict = {}
for file in fcc_files:
    with open(file) as f:
        file_encoding = f.encoding
    ##print(file)
    ## file.replace('../data/FCC_Broadband/used\\', '', 1).replace('.csv', '', 1)
    ##succinct = re.search(r"(?<!\d)\d{4}(?!\d)", file)
    succinct = re.search(r"(?<!\d)\d{4}(?!\d)", file)
    fcc_dict[succinct.group(0)] = pd.read_csv(file, encoding=file_encoding)

['../data/FCC_Broadband/TN_Only\\TN-Fixed-Dec2015-v4.csv', '../data/FCC_Broadband/TN_Only\\TN-Fixed-Dec2017-v3.csv', '../data/FCC_Broadband/TN_Only\\TN-Fixed-Dec2019-v1.csv']


In [57]:
## Import all files located in the data/FCC_Broadband/used folder adjacent to this directory
## Dict key - the unique part of the file name : Dict value - the dataframe
us_fcc_path = r'../data/FCC_Broadband/All_US' 
us_fcc_files = glob.glob(os.path.join(us_fcc_path, "*.csv"))
print(us_fcc_files)
us_fcc_dict = {}
for file in us_fcc_files:
    with open(file) as f:
        file_encoding = f.encoding

    ##print(file)
    ## file.replace('../data/FCC_Broadband/used\\', '', 1).replace('.csv', '', 1)
    ##succinct = re.search(r"(?<!\d)\d{4}(?!\d)", file)
    
    succinct = re.search(r"(?<!\d)\d{4}(?!\d)", file)
    us_fcc_dict[succinct.group(0)] = pd.read_csv(file, encoding=file_encoding)
for u in us_fcc_dict:
    print(us_fcc_dict[u].shape)

['../data/FCC_Broadband/All_US\\fbd_us_without_satellite_dec2015_v4.csv', '../data/FCC_Broadband/All_US\\fbd_us_without_satellite_dec2017_v3.csv', '../data/FCC_Broadband/All_US\\fbd_us_without_satellite_dec2019_v1.csv']
(24437598, 17)
(27890441, 17)
(30389413, 15)


In [4]:
# fcc_broadband_dec19 = pd.read_csv('../data/FCC_Broadband/TN-Fixed-Jun2019-v1.csv')
# fcc_broadband_dec17 = pd.read_csv('../data/FCC_Broadband/TN-Fixed-Dec2017-v3.csv')
# fcc_broadband_dec15 = pd.read_csv('../data/FCC_Broadband/TN-Fixed-Dec2015-v4.csv')
# replaced with a method of reading in files not dependent on file names

##For details on original columns: https://www.fcc.gov/general/explanation-broadband-deployment-data
# Code	Technology of Transmission
# 10	Asymmetric xDSL
# 11	ADSL2, ADSL2+
# 12	VDSL
# 20	Symmetric xDSL*
# 30	Other Copper Wireline (all copper-wire based technologies other than xDSL; Ethernet over copper and T-1 are examples)
# 40	Cable Modem other than DOCSIS 1, 1.1, 2.0, 3.0, or 3.1
# 41	Cable Modem – DOCSIS 1, 1.1 or 2.0
# 42	Cable Modem – DOCSIS 3.0
# 43	Cable Modem – DOCSIS 3.1
# 50	Optical Carrier / Fiber to the end user (Fiber to the home or business end user, does not include “fiber to the curb”)
# 60	Satellite
# 70	Terrestrial Fixed Wireless
# 90	Electric Power Line
# 0	All Other

## Urban Area definitions from 2018 Census Geographies 
## 'R' = Rural 'C' = Urban Cluster (2,500-50,000) 'U' = Urbanized Area (50,000+) 
urban_area_shape = gpd.read_file('../data/UrbanAreaShape/tl_2019_us_uac10.shp')

## Define central point of TN census tracts from 2010 Census Geographies
centroid_df = pd.read_csv('../data/CenPop2010_Mean_TR47.txt', 
                          converters={'TRACTCE': lambda x: str(x), 'COUNTYFP': lambda y: str(y)})

print(urban_area_shape.shape)
print(urban_area_shape.info())

print(centroid_df.shape)
print(centroid_df.info())

(3601, 13)
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3601 entries, 0 to 3600
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   UACE10      3601 non-null   object  
 1   GEOID10     3601 non-null   object  
 2   NAME10      3601 non-null   object  
 3   NAMELSAD10  3601 non-null   object  
 4   LSAD10      3601 non-null   object  
 5   MTFCC10     3601 non-null   object  
 6   UATYP10     3601 non-null   object  
 7   FUNCSTAT10  3601 non-null   object  
 8   ALAND10     3601 non-null   int64   
 9   AWATER10    3601 non-null   int64   
 10  INTPTLAT10  3601 non-null   object  
 11  INTPTLON10  3601 non-null   object  
 12  geometry    3601 non-null   geometry
dtypes: geometry(1), int64(2), object(10)
memory usage: 365.9+ KB
None
(1497, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------     

In [5]:
## TN Tracts shape
tract_shape = gpd.read_file('../data/shapefiles/tl_2019_47_tract.shp')

In [6]:
tract_shape.crs
tract_shape.head()
##tract_shape = tract_shape[['GEOID', 'geometry']]
tract_codes_only = tract_shape[['GEOID']]

In [7]:
tract_codes_only.rename(columns={'GEOID': 'TractCode'}, inplace=True)

C:\Users\Kyle\.conda\envs\geospatial\lib\site-packages\pandas\core\frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [8]:
urban_area_shape = urban_area_shape[['GEOID10', 'NAME10', 'UATYP10', 'ALAND10', 'geometry']]

In [9]:
## Current version of pres requires all dataframes run through 'refine' function which
## Adds Tract col
## Drops unneeded cols
## Remove satellite - techcode 60
## Adds a CountBlocks col which is the count of blocks in each tract
## Retains records marked 'Consumer' only

# def refine_frames(fcc_import_list):
#     out_frames = []
#     for df in fcc_import_list:
#         ## Add column for census tract
#         df['TractCode'] = df['BlockCode'].apply(lambda x: str(x)[0:11])
#         ## Drop extraneous columns
#         df = df.drop(['FRN', 'DBAName', 'HoldingCompanyName', 'HocoFinal', 'StateAbbr'], axis=1, inplace=False)
#         ## Remove any satellite records
#         df = df.loc[fcc_bbnd_june19['TechCode'] != 60]
#         ## Find how many census blocks are in each tract then merge those back in
#         count_blocks = df.groupby('TractCode')['BlockCode'].nunique().reset_index()
#         count_blocks.rename(columns={'BlockCode': 'CountBlocks'}, inplace=True)
#         merge_count = df.merge(count_blocks, how='inner', on='TractCode')
#         ## Limit to records with consumer service only
#         consumer_only =  merge_count[merge_count['Consumer'] == 1]
#         out_frames.append(consumer_only)        
#     return out_frames

def refine_frame(df):
    ## Add column for census tract
    df['TractCode'] = df['BlockCode'].apply(lambda x: str(x)[0:11])
    ## Drop extraneous columns
    df = df[['ProviderName', 'BlockCode', 'TechCode', 'Consumer', 'MaxAdDown', 'TractCode']]             
    ## Remove any satellite records
    df = df.loc[df['TechCode'] != 60]
    ## Find how many census blocks are in each tract then merge those back in
    count_blocks = df.groupby('TractCode')['BlockCode'].nunique().reset_index()
    count_blocks.rename(columns={'BlockCode': 'CountBlocks'}, inplace=True)
    merge_count = df.merge(count_blocks, how='inner', on='TractCode')
    ## Limit to records with consumer service only
    consumer_only =  merge_count[merge_count['Consumer'] == 1]
    print(consumer_only.shape)
    return consumer_only

def limit_by_consumer_dl_speeds(df, speed_limit):
    return df[df['MaxAdDown'] >= speed_limit]

In [10]:
# ## Find how many census blocks are in each tract
# count_blocks = no_sat_june19.groupby('TractCode')['BlockCode'].nunique().reset_index()
# count_blocks.rename(columns={'BlockCode': 'CountBlocks'}, inplace=True)

In [11]:
# ## Merge block count into fcc df
# no_sat_june19_countd = no_sat_june19.merge(count_blocks, how='inner', on='TractCode')

In [12]:
# ## Make dfs which are subsetted for Consumer and Business service, as well as both
# consumer_june19 =  no_sat_june19_countd[no_sat_june19_countd['Consumer'] == 1]
# business_june19 =  no_sat_june19_countd[no_sat_june19_countd['Business'] == 1]
# both_cnsm_biz_june19 = consumer_june19[consumer_june19['Business'] == 1]

# consumer_june19_25mbps = consumer_june19[consumer_june19['MaxAdDown'] >= 25.0]

In [95]:
## Create list of imported cleaned dataframes
for k in fcc_dict:
    fcc_dict[k] = refine_frame(fcc_dict[k])
    
# for k in fcc_dict:
#     print(fcc_dict[k].head())
fcc_dict_25 = copy.deepcopy(fcc_dict)
for m in fcc_dict_25:
    fcc_dict_25[m] = limit_by_consumer_dl_speeds(fcc_dict[m], 25)


(402030, 7)
(517453, 7)
(555957, 7)


In [58]:
for k in us_fcc_dict:
    us_fcc_dict[k] = refine_frame(us_fcc_dict[k])
us_fcc_dict_25 = copy.deepcopy(us_fcc_dict)
for m in us_fcc_dict_25:
    us_fcc_dict_25[m] = limit_by_consumer_dl_speeds(us_fcc_dict[m], 25)


(21714276, 7)
(25117432, 7)
(27754843, 7)


In [94]:
# test_coop = copy.deepcopy(fcc_dict)
# for t in test_coop:
#     coops = test_coop[t]
#     coops['Is Coop'] = coops['ProviderName'].str.contains('Coop', regex=False)
#     rslt_coops = coops.loc[coops['Is Coop'] == True, ['ProviderName']]
#     print(rslt_coops['ProviderName'].unique())

['Ben Lomand Rural Telephone Coop., Inc.'
 'Bledsoe Telephone Cooperative Inc'
 'Twin Lakes Telephone Cooperative Corporation'
 'North Central Telephone Cooperative, Inc.'
 'Scott County Telephone Cooperative'
 'West Kentucky Rural Telephone Coop Corp Inc']
['Ben Lomand Rural Telephone Coop., Inc.'
 'Twin Lakes Telephone Cooperative Corporation'
 'West Kentucky Rural Telephone Coop Corp Inc'
 'Bledsoe Telephone Cooperative Inc'
 'North Central Telephone Cooperative, Inc.'
 'Scott County Telephone Cooperative']
['Bledsoe Telephone Cooperative Inc'
 'Ben Lomand Rural Telephone Coop., Inc.'
 'West Kentucky Rural Telephone Coop Corp Inc'
 'Scott County Telephone Cooperative'
 'Twin Lakes Telephone Cooperative Corporation'
 'North Central Telephone Cooperative, Inc.'
 'Highland Telephone Cooperative']


In [66]:
combined_tech = pd.DataFrame()
for s in fcc_dict:
    avg_speed_by_tech = fcc_dict[s]
    avg_speed_by_tech['DSL'] = avg_speed_by_tech['TechCode'].isin(['10', '11', '12', '20'])
    avg_speed_by_tech['Fiber'] = avg_speed_by_tech['TechCode'].isin(['50'])
    avg_speed_by_tech['Fixed Terr'] = avg_speed_by_tech['TechCode'].isin(['70'])
    avg_speed_by_tech['Cable Other'] = avg_speed_by_tech['TechCode'].isin(['40', '41', '42'])
    avg_speed_by_tech['Cable DOCSIS 3.1'] = avg_speed_by_tech['TechCode'].isin(['43'])
    avg_speed_by_tech = avg_speed_by_tech.groupby(['DSL', 'Fiber', 'Fixed Terr', 'Cable Other', 'Cable DOCSIS 3.1'])['MaxAdDown'].mean().reset_index()
    avg_speed_by_tech['DSL'] = avg_speed_by_tech.loc[avg_speed_by_tech['DSL'] == True, ['MaxAdDown']]
    avg_speed_by_tech['Fiber'] = avg_speed_by_tech.loc[avg_speed_by_tech['Fiber'] == True, ['MaxAdDown']]
    avg_speed_by_tech['Fixed Terr'] = avg_speed_by_tech.loc[avg_speed_by_tech['Fixed Terr'] == True, ['MaxAdDown']]
    avg_speed_by_tech['Cable Other'] = avg_speed_by_tech.loc[avg_speed_by_tech['Cable Other'] == True, ['MaxAdDown']]
    avg_speed_by_tech['Cable DOCSIS 3.1'] = avg_speed_by_tech.loc[avg_speed_by_tech['Cable DOCSIS 3.1'] == True, ['MaxAdDown']]
    avg_speed_by_tech = avg_speed_by_tech.drop('MaxAdDown', axis=1, inplace=False)
    combined_tech[f'{s}'] = avg_speed_by_tech.max()
combined_tech = combined_tech.transpose().fillna(0)
combined_tech.to_csv(f'../out/speed_by_tech.csv')

In [11]:
## Define the tracts which are minimally served by definition, given they are in the '98 - 99' range of tracts

def find_na_tracts(df_dict):
    ret_dict = {}
    for c in df_dict:
        na_tracts = df_dict[c]
        na_tracts['Minimally Served by Definition'] = na_tracts['TractCode'].map(lambda x: x[5:7] in ['98', '99'])        
        na_tracts = na_tracts[['Minimally Served by Definition', 'TractCode']]
        na_tracts = na_tracts.drop_duplicates()
        na_tracts.to_csv(f'../out/def_minimal_service_{c.lower()}.csv')
        ret_dict[c] = na_tracts
    return ret_dict

In [12]:
find_na_tracts(fcc_dict)

{'2015':         Minimally Served by Definition    TractCode
 1                                False  47093004611
 258                              False  47037015900
 497                              False  47143975401
 1320                             False  47093000100
 1663                             False  47093000902
 ...                                ...          ...
 421678                            True  47161980100
 421682                           False  47005963000
 421859                           False  47121960100
 422023                            True  47009980200
 422028                            True  47017980100
 
 [1495 rows x 2 columns],
 '2017':         Minimally Served by Definition    TractCode
 13                               False  47037019500
 1860                             False  47001020100
 2101                             False  47001020201
 2229                             False  47001020202
 2541                             False  47001020902
 .

In [65]:
merge_tracts_speed = copy.deepcopy(tract_codes_only)

In [59]:
def avg_servable_speed_out(df_dict, merge_tracts, suffix='tn', min_speed=0):    
    for c in df_dict:
        max_speed_per_block = df_dict[c].groupby(['BlockCode', 'TractCode'])['MaxAdDown'].max().reset_index()
        max_speed_per_block = max_speed_per_block[max_speed_per_block['MaxAdDown'] > 0.0]
        avg_servable_speed = max_speed_per_block.groupby('TractCode')['MaxAdDown'].mean().reset_index()
        avg_servable_speed = avg_servable_speed.add_suffix(f'_{c}')        
        merge_tracts = merge_tracts.merge(avg_servable_speed, how='left', left_on='TractCode', right_on=f'TractCode_{c}')
        merge_tracts.drop([f'TractCode_{c}'], axis=1, inplace=True)
        print(merge_tracts.shape)
    merge_tracts.to_csv(f'../out/avg_servable_speed_all_years_{suffix}.csv')    
    return merge_tracts    

In [77]:
def avg_servable_speed_us(df_dict, suffix='us', min_speed=0):    
    all_tracts = pd.Series()
    for c in df_dict:
        current_tracts = pd.Series(df_dict[c]['TractCode'].unique())        
        all_tracts = pd.concat([current_tracts, all_tracts], axis=0)
        
    merge_tracts_df = pd.DataFrame(data=all_tracts, columns=['TractCode'])
    
    for c in df_dict:
        max_speed_per_block = df_dict[c].groupby(['BlockCode', 'TractCode'])['MaxAdDown'].max().reset_index()
        max_speed_per_block = max_speed_per_block[max_speed_per_block['MaxAdDown'] > 0.0]
        avg_servable_speed = max_speed_per_block.groupby('TractCode')['MaxAdDown'].mean().reset_index()
        avg_servable_speed = avg_servable_speed.add_suffix(f'_{c}')        
        merge_tracts_df = merge_tracts_df.merge(avg_servable_speed, how='left', left_on='TractCode', right_on=f'TractCode_{c}')
        merge_tracts_df.drop([f'TractCode_{c}'], axis=1, inplace=True)
        print(merge_tracts_df.shape)
    merge_tracts_df.to_csv(f'../out/avg_servable_speed_all_years_{suffix}.csv')    
    return merge_tracts_df

In [78]:
avg_servable_speed_us(us_fcc_dict)

<ipython-input-77-c6f9f3dc58c4>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  all_tracts = pd.Series()


(299239, 2)
(299239, 3)
(299239, 4)


,TractCode,MaxAdDown_2015,MaxAdDown_2017,MaxAdDown_2019
0,56037970500,49.857778,33.675620,137.515373
1,56037971600,50.221076,13.518421,38.267724
2,49009960100,1.992891,6.957031,72.170588
3,49013940300,205.719222,138.320804,158.615728
4,49013940600,105.938596,113.892826,148.188730
...,...,...,...,...
299234,48473680400,3.000000,19.800000,509.500000
299235,47005963000,11.797957,15.927788,46.562000
299236,47121960100,9.693692,243.851468,306.225904
299237,47009980200,1.814400,25.000000,6.410959


In [66]:
avg_servable_speed_out(fcc_dict, merge_tracts_speed)

(1497, 2)
(1497, 3)
(1497, 4)


,TractCode,MaxAdDown_2015,MaxAdDown_2017,MaxAdDown_2019
0,47165021002,216.792386,963.500000,950.086957
1,47165020901,179.441333,772.750000,829.073667
2,47165020300,220.146789,865.549550,887.648649
3,47165020902,277.924724,924.168142,952.920354
4,47165020800,344.770992,921.820896,955.947761
...,...,...,...,...
1492,47157022112,141.761194,140.117647,903.115942
1493,47073050501,87.245902,298.093333,710.518072
1494,47073050601,97.104972,394.519337,931.010989
1495,47073050602,100.000000,400.000000,941.452174


In [19]:
merge_tracts_choice = copy.deepcopy(tract_codes_only)

In [20]:
# ## Determine how many blocks in a tract have a *choice of services with multiple providers* at levels of any / 25 mbps

# def choice_levels_out(df_dict, merge_tracts, min_speed=0):
#     s = ('_' + str(min_speed) + 'mbps') if min_speed > 0 else ''
#     ## ret_dict = {}
#     for c in df_dict:
#         group_choice = df_dict[c].groupby(['ProviderName','BlockCode', 'TractCode', 'CountBlocks'])['Consumer'].sum().reset_index()
#         group_choice = group_choice[group_choice['Consumer'] >= 2]
#         group_choice_counted = group_choice.groupby(['TractCode', 'CountBlocks']).count().reset_index()       
#         group_choice_counted.drop(['ProviderName', 'BlockCode'], axis=1, inplace=True)
#         group_choice_counted.rename(columns={'Consumer': 'Blocks With Choice'}, inplace=True)
#         group_choice_counted['Prop With Choice'] = round((100 * group_choice_counted['Blocks With Choice'] / group_choice_counted['CountBlocks']), 2)
#         group_choice_counted = group_choice_counted.fillna(0)
#         group_choice_counted = group_choice_counted.add_suffix(f'_{c}')
#         merge_tracts = merge_tracts.merge(group_choice_counted, how='left', left_on='TractCode', right_on=f'TractCode_{c}')
#         merge_tracts.drop([f'TractCode_{c}'], axis=1, inplace=True)
#     merge_tracts.to_csv(f'../out/choice_levels_all_years{s}.csv')
#     return merge_tracts

In [21]:
## Determine how many blocks in a tract have a *choice of services with multiple providers* at levels of any / 25 mbps

def choice_levels_out(df_dict, merge_tracts, min_speed=0):
    s = ('_' + str(min_speed) + 'mbps') if min_speed > 0 else ''
    ## ret_dict = {}
    for c in df_dict:
        group_choice = df_dict[c].groupby(['ProviderName','BlockCode', 'TractCode', 'CountBlocks'])['Consumer'].max().reset_index()
        group_choice_counted = group_choice.groupby(['BlockCode', 'TractCode', 'CountBlocks'])['Consumer'].sum().reset_index()
        group_choice_counted = group_choice_counted[group_choice_counted['Consumer'] >= 2]
        group_choice_counted = group_choice_counted.groupby(['TractCode', 'CountBlocks']).count().reset_index()
        group_choice_counted.drop(['BlockCode'], axis=1, inplace=True)
        group_choice_counted.rename(columns={'Consumer': 'Blocks With Choice'}, inplace=True)
        group_choice_counted['Prop With Choice'] = round((100 * group_choice_counted['Blocks With Choice'] / group_choice_counted['CountBlocks']), 2)
        group_choice_counted = group_choice_counted.fillna(0)
        group_choice_counted = group_choice_counted.add_suffix(f'_{c}')
        merge_tracts = merge_tracts.merge(group_choice_counted, how='left', left_on='TractCode', right_on=f'TractCode_{c}')
        merge_tracts.drop([f'TractCode_{c}'], axis=1, inplace=True)
    merge_tracts.to_csv(f'../out/choice_levels_all_years{s}.csv')
    return merge_tracts

In [22]:
choice_levels_out(fcc_dict, merge_tracts_choice)
choice_levels_out(fcc_dict_25, merge_tracts_choice, 25)

,TractCode,CountBlocks_2015,Blocks With Choice_2015,Prop With Choice_2015,CountBlocks_2017,Blocks With Choice_2017,Prop With Choice_2017,CountBlocks_2019,Blocks With Choice_2019,Prop With Choice_2019
0,47165021002,84.0,8.0,9.52,73.0,62.0,84.93,73.0,61.0,83.56
1,47165020901,NaN,NaN,NaN,25.0,14.0,56.00,25.0,14.0,56.00
2,47165020300,113.0,4.0,3.54,111.0,42.0,37.84,111.0,77.0,69.37
3,47165020902,116.0,15.0,12.93,114.0,97.0,85.09,113.0,99.0,87.61
4,47165020800,134.0,24.0,17.91,134.0,85.0,63.43,134.0,114.0,85.07
...,...,...,...,...,...,...,...,...,...,...
1492,47157022112,67.0,4.0,5.97,68.0,44.0,64.71,69.0,45.0,65.22
1493,47073050501,61.0,15.0,24.59,75.0,17.0,22.67,83.0,28.0,33.73
1494,47073050601,181.0,55.0,30.39,181.0,65.0,35.91,182.0,40.0,21.98
1495,47073050602,115.0,28.0,24.35,115.0,39.0,33.91,115.0,34.0,29.57


In [98]:
merge_tracts_per_provider = copy.deepcopy(tract_codes_only)

In [99]:
municipal_providers = ['Athens Utility Board', 'BRISTOL TENNESSEE ESSENTIAL SERVICES', 
                       'BVU Authority', 'Johnson City Energy Authority dba BrightRidge',
                       'City of Maryville', 'Clarksville CDE Lightband', 'Columbia Power and Water System',
                       'EPB Fiber Optics', 'Erwin Utilities', 'Fayetteville Public Utilities', 'Holston Connect, LLC',
                       'Jackson Energy Authority', 'Lenoir City Utilities Board', 'MPU Connect', 'NESNet',
                       'Newport Utilities', 'Pulaski Electric System', 'SVE Connect, LLC',
                       'TRI-COUNTY FIBER COMMUNICATIONS, LLC', 'Tullahoma Utilities Authority']

cooperatives = ['Ben Lomand Rural Telephone Coop., Inc.', 'Bledsoe Telephone Cooperative Inc',
                'Twin Lakes Telephone Cooperative Corporation', 'North Central Telephone Cooperative, Inc.',
                'Scott County Telephone Cooperative', 'West Kentucky Rural Telephone Coop Corp Inc',
               'Highland Telephone Cooperative']



def blocks_per_provider_out(df_dict, merge_tracts, min_speed=0):
    s = ('_' + str(min_speed) + 'mbps') if min_speed > 0 else ''
    
    all_providers = pd.Series()    
    for c in df_dict:
        current_providers = pd.Series(df_dict[c]['ProviderName'].unique())        
        all_providers = pd.concat([current_providers, all_providers], axis=0)    
    
    provider_tracts = pd.DataFrame(list(itertools.product(all_providers, merge_tracts['TractCode'])), columns=['ProviderName','TractCode'])    
    
    for c in df_dict:
        provider_blocks = df_dict[c].groupby(['ProviderName', 'BlockCode', 'TractCode', 'CountBlocks'])['Consumer'].max().reset_index()
        total_provider_blocks = provider_blocks.groupby(['ProviderName','TractCode', 'CountBlocks'])['Consumer'].sum().reset_index()
        total_provider_blocks.rename(columns={'Consumer': 'Provider Blocks Served'}, inplace=True)     
        total_provider_blocks['Prop With Provider'] = round((100 * total_provider_blocks['Provider Blocks Served'] / total_provider_blocks['CountBlocks']), 2)
        total_provider_blocks['Substantially Served by Provider'] = total_provider_blocks['Prop With Provider'] >= 5
        total_provider_blocks = total_provider_blocks.fillna(0)
        total_provider_blocks = total_provider_blocks.add_suffix(f'_{c}')
        print(total_provider_blocks.info())
        provider_tracts = provider_tracts.merge(total_provider_blocks, how='left', left_on=['TractCode', 'ProviderName'], right_on=[f'TractCode_{c}' , f'ProviderName_{c}'])
        provider_tracts.drop([f'TractCode_{c}', f'ProviderName_{c}'], axis=1, inplace=True)
    
    provider_tracts['Is Co-op'] = provider_tracts['ProviderName'].isin(cooperatives)
    provider_tracts['Is Municipal'] = provider_tracts['ProviderName'].isin(municipal_providers)
    provider_tracts.drop_duplicates(inplace=True)
    provider_tracts.to_csv(f'../out/blocks_per_provider_all_years{s}.csv')    
    return provider_tracts

In [100]:
blocks_per_provider_out(fcc_dict, merge_tracts_per_provider)
blocks_per_provider_out(fcc_dict_25, merge_tracts_per_provider, 25)

<ipython-input-99-e2fa3a5b4cff>:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  all_providers = pd.Series()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4834 entries, 0 to 4833
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ProviderName_2015                      4834 non-null   object 
 1   TractCode_2015                         4834 non-null   object 
 2   CountBlocks_2015                       4834 non-null   int64  
 3   Provider Blocks Served_2015            4834 non-null   int64  
 4   Prop With Provider_2015                4834 non-null   float64
 5   Substantially Served by Provider_2015  4834 non-null   bool   
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 193.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5070 entries, 0 to 5069
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ProviderName_2017        

<ipython-input-99-e2fa3a5b4cff>:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  all_providers = pd.Series()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3407 entries, 0 to 3406
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ProviderName_2015                      3407 non-null   object 
 1   TractCode_2015                         3407 non-null   object 
 2   CountBlocks_2015                       3407 non-null   int64  
 3   Provider Blocks Served_2015            3407 non-null   int64  
 4   Prop With Provider_2015                3407 non-null   float64
 5   Substantially Served by Provider_2015  3407 non-null   bool   
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 136.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3877 entries, 0 to 3876
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ProviderName_2017        

,ProviderName,TractCode,CountBlocks_2015,Provider Blocks Served_2015,Prop With Provider_2015,Substantially Served by Provider_2015,CountBlocks_2017,Provider Blocks Served_2017,Prop With Provider_2017,Substantially Served by Provider_2017,CountBlocks_2019,Provider Blocks Served_2019,Prop With Provider_2019,Substantially Served by Provider_2019,Is Co-op,Is Municipal
0,"COMCAST CABLE COMMUNICATIONS, LLC",47165021002,83.0,71.0,85.54,True,68.0,65.0,95.59,True,69.0,65.0,94.20,True,False,False
1,"COMCAST CABLE COMMUNICATIONS, LLC",47165020901,21.0,14.0,66.67,True,24.0,18.0,75.00,True,24.0,18.0,75.00,True,False,False
2,"COMCAST CABLE COMMUNICATIONS, LLC",47165020300,109.0,95.0,87.16,True,111.0,97.0,87.39,True,111.0,97.0,87.39,True,False,False
3,"COMCAST CABLE COMMUNICATIONS, LLC",47165020902,116.0,104.0,89.66,True,113.0,104.0,92.04,True,113.0,107.0,94.69,True,False,False
4,"COMCAST CABLE COMMUNICATIONS, LLC",47165020800,131.0,123.0,93.89,True,134.0,122.0,91.04,True,134.0,122.0,91.04,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296401,"Atlantic Broadband Finance, LLC",47157022112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
296402,"Atlantic Broadband Finance, LLC",47073050501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
296403,"Atlantic Broadband Finance, LLC",47073050601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
296404,"Atlantic Broadband Finance, LLC",47073050602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [37]:
merge_tracts_blocks_served = copy.deepcopy(tract_codes_only)

In [38]:
def blocks_served_anyone_out(df_dict, merge_tracts, min_speed=0):
    s = ('_' + str(min_speed) + 'mbps') if min_speed > 0 else ''
    ##ret_dict = {}
    for c in df_dict:
        served_by_any_blocks = df_dict[c].groupby(['BlockCode', 'TractCode', 'CountBlocks'])['Consumer'].max().reset_index()
        served_by_any_blocks = served_by_any_blocks.groupby(['TractCode', 'CountBlocks'])['Consumer'].sum().reset_index()
        served_by_any_blocks.rename(columns={'Consumer': 'Blocks Served Any Provider'}, inplace=True)       
        served_by_any_blocks['Prop Served Any'] = round((100 * served_by_any_blocks['Blocks Served Any Provider'] / served_by_any_blocks['CountBlocks']), 2)
        served_by_any_blocks = served_by_any_blocks.fillna(0)
        served_by_any_blocks = served_by_any_blocks.add_suffix(f'_{c}')
        merge_tracts = merge_tracts.merge(served_by_any_blocks, how='left', left_on='TractCode', right_on=f'TractCode_{c}')
        merge_tracts.drop([f'TractCode_{c}'], axis=1, inplace=True)       
     
    merge_tracts.to_csv(f'../out/blocks_served_any_all_years{s}.csv')        
    return merge_tracts

In [39]:
blocks_served_anyone_out(fcc_dict, merge_tracts_blocks_served)
blocks_served_anyone_out(fcc_dict_25, merge_tracts_blocks_served, 25)

,TractCode,CountBlocks_2015,Blocks Served Any Provider_2015,Prop Served Any_2015,CountBlocks_2017,Blocks Served Any Provider_2017,Prop Served Any_2017,CountBlocks_2019,Blocks Served Any Provider_2019,Prop Served Any_2019
0,47165021002,84.0,74.0,88.10,73.0,66.0,90.41,73.0,67.0,91.78
1,47165020901,22.0,17.0,77.27,25.0,23.0,92.00,25.0,23.0,92.00
2,47165020300,113.0,95.0,84.07,111.0,101.0,90.99,111.0,104.0,93.69
3,47165020902,116.0,107.0,92.24,114.0,111.0,97.37,113.0,111.0,98.23
4,47165020800,134.0,125.0,93.28,134.0,130.0,97.01,134.0,133.0,99.25
...,...,...,...,...,...,...,...,...,...,...
1492,47157022112,67.0,63.0,94.03,68.0,64.0,94.12,69.0,64.0,92.75
1493,47073050501,61.0,53.0,86.89,75.0,71.0,94.67,83.0,77.0,92.77
1494,47073050601,181.0,176.0,97.24,181.0,180.0,99.45,182.0,181.0,99.45
1495,47073050602,115.0,115.0,100.00,115.0,115.0,100.00,115.0,115.0,100.00


In [13]:
merge_tracts_tech = copy.deepcopy(tract_codes_only)

In [59]:
def tech_available_out(df_dict, merge_tracts, min_speed=0):
    s = ('_' + str(min_speed) + 'mbps') if min_speed > 0 else ''
    
    for c in df_dict:        
        print(df_dict[c]['TechCode'].unique())
        df_dict[c]['TechCode'] = df_dict[c]['TechCode'].map({10: 10, 11: 10, 12: 10, 20: 10, 
                                                             30: 30, 40: 42, 41: 42, 42: 42, 43: 43, 50: 50, 70: 70, 90: 90})
        print(df_dict[c]['TechCode'].unique())
    
    all_tech = pd.Series(dtype='int64')
    for c in df_dict:
        current_tech = pd.Series(df_dict[c]['TechCode'].unique())        
        all_tech = pd.concat([current_tech, all_tech], axis=0)    
    
    tech_tracts = pd.DataFrame(list(itertools.product(all_tech, merge_tracts['TractCode'])), columns=['TechCode','TractCode'])
    
    for c in df_dict:
        tech_available_blocks = df_dict[c].groupby(['TechCode', 'BlockCode', 'TractCode', 'CountBlocks'])['Consumer'].max().reset_index()
        tech_available_blocks = tech_available_blocks.groupby(['TechCode', 'TractCode', 'CountBlocks'])['Consumer'].sum().reset_index()
        tech_available_blocks.rename(columns={'Consumer': 'Blocks Served Using Tech'}, inplace=True)       
        tech_available_blocks['Prop With Tech'] = round((100 * tech_available_blocks['Blocks Served Using Tech'] / tech_available_blocks['CountBlocks']), 2)
        tech_available_blocks['Tech Available'] = tech_available_blocks['Prop With Tech'] >= 5
        tech_available_blocks = tech_available_blocks.fillna(0)
        tech_available_blocks = tech_available_blocks.add_suffix(f'_{c}')
        tech_tracts = tech_tracts.merge(tech_available_blocks, how='left', left_on=['TractCode', 'TechCode'], right_on=[f'TractCode_{c}', f'TechCode_{c}'])
        tech_tracts.drop([f'TractCode_{c}'], axis=1, inplace=True)
        
    tech_tracts.drop_duplicates(inplace=True)   
    tech_tracts.to_csv(f'../out/tech_available_all_years{s}.csv')        
    return tech_tracts

In [60]:
statewide_percent = tech_available_out(fcc_dict, merge_tracts_tech)
tech_available_out(fcc_dict_25, merge_tracts_tech, 25)

[42 12 11 50 10 30 70 20 40 41]
[42 10 50 30 70]
[10 11 12 50 42 30 43 70 41 40 20]
[10 50 42 30 43 70]
[70 43 10 11 12 50 42 30 40]
[70 43 10 50 42 30]
[42 12 11 50 70 40]
[42 10 50 70]
[11 12 50 42 30 43 70 40 10 41]
[10 50 42 30 43 70]
[43 11 12 50 70 42 30 40 10]
[43 10 50 70 42 30]


<ipython-input-59-b60d9f5e4fa0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict[c]['TechCode'] = df_dict[c]['TechCode'].map({10: 10, 11: 10, 12: 10, 20: 10,


,TechCode,TractCode,TechCode_2015,CountBlocks_2015,Blocks Served Using Tech_2015,Prop With Tech_2015,Tech Available_2015,TechCode_2017,CountBlocks_2017,Blocks Served Using Tech_2017,Prop With Tech_2017,Tech Available_2017,TechCode_2019,CountBlocks_2019,Blocks Served Using Tech_2019,Prop With Tech_2019,Tech Available_2019
0,43,47165021002,NaN,NaN,NaN,NaN,NaN,43.0,73.0,65.0,89.04,True,43.0,73.0,65.0,89.04,True
1,43,47165020901,NaN,NaN,NaN,NaN,NaN,43.0,25.0,18.0,72.00,True,43.0,25.0,20.0,80.00,True
2,43,47165020300,NaN,NaN,NaN,NaN,NaN,43.0,111.0,97.0,87.39,True,43.0,111.0,97.0,87.39,True
3,43,47165020902,NaN,NaN,NaN,NaN,NaN,43.0,114.0,104.0,91.23,True,43.0,113.0,107.0,94.69,True
4,43,47165020800,NaN,NaN,NaN,NaN,NaN,43.0,134.0,122.0,91.04,True,43.0,134.0,122.0,91.04,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8977,30,47157022112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8978,30,47073050501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8979,30,47073050601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8980,30,47073050602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
tech_2015 = pd.DataFrame(100 * (statewide_percent.groupby('TechCode')['Tech Available_2015'].count() / statewide_percent['TractCode'].nunique()))
tech_2017 = pd.DataFrame(100 * (statewide_percent.groupby('TechCode')['Tech Available_2017'].count() / statewide_percent['TractCode'].nunique()))
tech_2019 = pd.DataFrame(100 * (statewide_percent.groupby('TechCode')['Tech Available_2019'].count() / statewide_percent['TractCode'].nunique()))

tech_pct_all_years = pd.concat([tech_2015, tech_2017, tech_2019], axis=1)
tech_pct_date_top = tech_pct_all_years.transpose()

tech_pct_date_top.rename({10: 'Any DSL', 30: 'Other Copper', 42: 'Cable Modem Other', 43: 'Cable Modem DOCSIS 3.1', 50 : 'Optical Carrier Fiber',
                          70: 'Terrestrial Fixed Wireless', 90: 'Electric Power Line'}, axis=1, inplace=True)
tech_pct_date_top.rename({'Tech Available_2015': '2015', 'Tech Available_2017': '2017', 'Tech Available_2019': '2019'}, axis=0, inplace=True)
tech_pct_date_top.to_csv('../out/tech_pct_statewide_all_years.csv')

In [ ]:
census_url = 'https://api.census.gov/data'
commute_years = ['2015', '2017', '2019']
commute_dataset = 'acs/acs5'
commute_base_urls = {}
commute_dict = {}
for y in commute_years:
    commute_base_urls[y] = '/'.join([census_url, y, commute_dataset])
get_commute_vars = ['B08303_' + str(i + 1).zfill(3) + "E" for i in range(0, 13)]
get_wfh_vars = ['B08301_001E', 'B08301_021E']
get_tract_vars = ['NAME'] + get_wfh_vars + get_commute_vars
predicates = {}
predicates['get'] = ','.join(get_tract_vars)
predicates['for'] = 'tract:*'
predicates['in'] = 'state:47'
for y in commute_base_urls:
    commute_dict[y] = requests.get(commute_base_urls[y], params=predicates)

col_names = ['NAME', 'Workers - Commute Method', 'Work From Home', 
             'Workers - Commute Length', 'Less Than 5', '5 to 9', '10 to 14', '15 to 19', '20 to 24', 
             '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 59', '60 to 89', '90 plus',
             'state', 'county', 'tract']
commute_times = [4, 9, 14, 19, 24, 29, 34, 39, 44, 59, 89, 120]

merge_commute_tracts = copy.deepcopy(tract_codes_only)

In [ ]:
for y in commute_dict:
    commute_df = pd.DataFrame(columns=col_names, data=commute_dict[y].json()[1:])    
    reduced_commute_df = commute_df.drop(['NAME', 'Workers - Commute Method', 'Work From Home', 'Workers - Commute Length', 
                     'state', 'county', 'tract'], axis='columns')
    reduced_commute_int = reduced_commute_df.astype('int64', copy=True)    
    mul_commute = reduced_commute_int * commute_times    
    sr = round((mul_commute.sum(axis=1) / reduced_commute_int.sum(axis=1)), 2).rename('Median Commute Length').fillna(0)
    wfh_commute_simple = commute_df.merge(sr, left_index=True, right_index=True)
    wfh_commute_simple = wfh_commute_simple.astype({'Workers - Commute Method':'int64', 'Work From Home': 'int64'})
    wfh_commute_simple['TractCode'] = wfh_commute_simple['state'] + wfh_commute_simple['county'] + wfh_commute_simple['tract']
    wfh_commute_simple['Proportion Work From Home'] = round(100 * (wfh_commute_simple['Work From Home'] / wfh_commute_simple['Workers - Commute Method']), 2)
    wfh_commute_reduced = wfh_commute_simple[['NAME', 'Workers - Commute Method', 
                                          'Work From Home', 'Proportion Work From Home', 
                                          'Workers - Commute Length', 'Median Commute Length','TractCode']]
    wfh_commute_reduced = wfh_commute_reduced.add_suffix(f'_{y}')
    merge_commute_tracts = merge_commute_tracts.merge(wfh_commute_reduced, how='left', left_on='TractCode', right_on=f'TractCode_{y}')
    merge_commute_tracts.drop([f'TractCode_{y}'], axis=1, inplace=True)
    
merge_commute_tracts.to_csv(f'../out/wfh_commute_all_years.csv')

In [ ]:
ed_dataset = 'acs/acs5'
ed_years = ['2015', '2017', '2019']
ed_base_urls = {}
ed_dict = {}
for y in ed_years:
    ed_base_urls[y] = '/'.join([census_url, y, ed_dataset])
get_vars_ed = ['B15003_' + str(i + 1).zfill(3) + 'E' for i in range(0, 25)]
get_vars_ed = ['NAME'] + get_vars_ed
predicates_ed = {}
predicates_ed['get'] = ','.join(get_vars_ed)
predicates_ed['for'] = 'tract:*'
predicates_ed['in']  = 'state:47'  ##State of Tennessee
for y in ed_base_urls:
    ed_dict[y] = requests.get(ed_base_urls[y], params=predicates_ed)

ed_col_names = ['NAME', 'Total', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014',
             '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', 'state', 'county', 'tract']
merge_ed_tracts = copy.deepcopy(tract_codes_only)

for y in ed_dict:
    ed_df = pd.DataFrame(columns=ed_col_names, data=ed_dict[y].json()[1:])
    ints_dict = {}
    int_cols = [e for e in ed_col_names if e not in ('NAME', 'state', 'county', 'tract')]
    for f in int_cols:
        ints_dict[f] = 'int'
    ed_df = ed_df.astype(ints_dict)  ##Change measures to be ints so they can be used to calculate
    
    running = pd.DataFrame()
    
    ##Add Summary columns
    running = ed_df[[str(i).zfill(3) for i in range(2, 17)]]    
    ed_df['Less_Than_High_School'] = running.sum(axis=1)
    running = ed_df[['017', '018']]
    ed_df['High_School_Or_GED'] = running.sum(axis=1)
    running = ed_df[['019', '020', '021']]
    ed_df['Some_College_Or_Associates'] = running.sum(axis=1)
    running = ed_df[['022', '023', '024', '025']]
    ed_df['Bachelors_Or_More'] = running.sum(axis=1)

    ##Create Proportion columns
    ed_df['Less_HS_Proportion'] = round((100 * ed_df['Less_Than_High_School'] / ed_df['Total']), 2)
    ed_df['HS_Proportion'] = round((100 * ed_df['High_School_Or_GED'] / ed_df['Total']), 2)
    ed_df['Some_College_Proportion'] = round((100 * ed_df['Some_College_Or_Associates'] / ed_df['Total']), 2)
    ed_df['Bachelors_Proportion'] = round((100 * ed_df['Bachelors_Or_More'] / ed_df['Total']), 2)
    
    ed_df['TractCode'] = ed_df['state'] + ed_df['county'] + ed_df['tract']
    ed_df = ed_df[['TractCode', 'Less_HS_Proportion', 'HS_Proportion', 'Some_College_Proportion', 'Bachelors_Proportion']].fillna(0)    
    
    ed_df = ed_df.add_suffix(f'_{y}')
    merge_ed_tracts = merge_ed_tracts.merge(ed_df, how='left', left_on='TractCode', right_on=f'TractCode_{y}')
    merge_ed_tracts.drop([f'TractCode_{y}'], axis=1, inplace=True)
    
merge_ed_tracts.to_csv(f'../out/ed_levels_all_years.csv')
    

In [ ]:
# map_top_providers_dfs = []
# top_n_providers = list(consumer_set_codes.groupby('ProviderName').size().sort_values().tail(10).index)
# rp = copy.copy(cm.get_cmap('tab20c'))
# for x in top_n_providers:
#     provider_sub = consumer_set_codes[consumer_set_codes['ProviderName'] == x]
#     mappable = tract_shape.merge(provider_sub, how='left', left_on='GEOID', right_on='TractCode')
#     map_top_providers_dfs.append(mappable)
# for provider_map in map_top_providers_dfs:
#     provider_map.plot(column='Substantially Served', cmap=rp)

In [ ]:
centroid_df.drop('POPULATION', axis=1, inplace=True)

In [ ]:
centroid_df = centroid_df.astype({'STATEFP': 'str'}, copy=False)

In [ ]:
centroid_df['TractCode'] = centroid_df['STATEFP'] + centroid_df['COUNTYFP'] + centroid_df['TRACTCE']

In [ ]:
centroid_df['geometry'] = centroid_df.apply(lambda x: Point((float(x['LONGITUDE']), float(x['LATITUDE']))), axis=1)

In [ ]:
centroid_geo = gpd.GeoDataFrame(centroid_df, crs = urban_area_shape.crs, geometry = centroid_df['geometry'])
centroid_geo = centroid_geo[['LATITUDE', 'LONGITUDE', 'TractCode', 'geometry']]

In [ ]:
ua_of_census_tracts = gpd.sjoin(centroid_geo, urban_area_shape, op = 'within')

In [ ]:
ua_of_census_tracts.drop('index_right', axis=1, inplace=True)

In [ ]:
##Find any rows where Tract is not in the spatially merged df
centroid_geo_rural = centroid_geo[centroid_geo['TractCode'].isin(ua_of_census_tracts['TractCode'].tolist()) == False]

In [ ]:
## UATYP10 is 'R' for Rural, 'U' for Urbanized Area (50,000+), and 'C' for Urban Cluster (2,500 - 50,000).

centroid_rural_match_cols = pd.concat([centroid_geo_rural, pd.DataFrame(columns=['GEOID10', 'NAME10', 'UATYP10', 'ALAND10'])])
centroid_rural_match_cols['UATYP10'] = 'R'
centroid_rural_match_cols['ALAND10'] = centroid_rural_match_cols['ALAND10'].fillna(np.nan)
centroid_rural_match_cols['NAME10'] = centroid_rural_match_cols['NAME10'].fillna('Rural Area')

In [ ]:
tract_by_ua_w_rural = pd.concat([centroid_rural_match_cols, ua_of_census_tracts], axis=0)

In [ ]:
tract_by_ua_w_rural['UATYP10'].unique()
tract_by_ua_w_rural.to_csv('../out/tracts_urban_def.csv')

In [ ]:
# census_url = 'https://api.census.gov/data'
# year = '2010'
# dataset = 'dec/sf1'
# base_url = '/'.join([census_url, year, dataset])
# get_tract_vars = ['P001001']
# get_tract_vars = ["NAME"] + get_tract_vars
# predicates = {}
# predicates['get'] = ','.join(get_tract_vars)
# predicates['for'] = 'urban area:*'  ## zctas
# ##predicates['in'] = 'state:47'
# request = requests.get(base_url, params=predicates)
# ##col_names = ['NAME', 'Population', 'zip']
# census_df = pd.DataFrame(data=request.json()[1:])

# ##https://api.census.gov/data/2010/dec/sf1?get=P001001,NAME&for=urban%20rural:*&in=state:01%20county:087%20county%20subdivision:93230%20place/remainder%20(or%20part):77304%20tract%20(or%20part):231900

In [ ]:
census_url = 'https://api.census.gov/data'
## Assumed that the years are entered in chron order
pop_years = ['2015', '2019']
pop_dataset = 'acs/acs5'
pop_base_urls = {}
pop_dict = {}
for y in pop_years:
    pop_base_urls[y] = '/'.join([census_url, y, pop_dataset])

get_pop_tract_vars = ['B01003_001E']
get_pop_tract_vars = ["NAME"] + get_pop_tract_vars
pop_predicates = {}
pop_predicates['get'] = ','.join(get_pop_tract_vars)
pop_predicates['for'] = 'tract:*'
pop_predicates['in'] = 'state:47'
pop_requests = []
for y in pop_base_urls:
    pop_dict[y] = requests.get(pop_base_urls[y], params=pop_predicates)
col_names = ['NAME', 'Population', 'state', 'county', 'tract']
for y in pop_dict:
    population_df = pd.DataFrame(columns=col_names, data=pop_dict[y].json()[1:])
    population_df['TractCode'] = population_df['state'] + population_df['county'] + population_df['tract']
    population_df = population_df.astype({'Population': 'int64'})
    population_df = population_df[['NAME', 'Population', 'TractCode']].sort_values('TractCode')
    pop_dict[y] = population_df    
list(pop_dict.values())[0]

first_year_pop_df = list(pop_dict.values())[0]
last_year_pop_df = list(pop_dict.values())[-1]
increase_pop_df = first_year_pop_df.merge(last_year_pop_df, how='inner', on='TractCode', suffixes=('_first', '_last'))

increase_pop_df['Pct Pop Change'] = 100 * round(((increase_pop_df['Population_last'] - increase_pop_df['Population_first']) / increase_pop_df['Population_first']), 2)
increase_pop_df.to_csv('../out/pop_change.csv')